In [1]:
!pip install datasets>=1.18.3
!pip install transformers==4.11.3
!pip install librosa
!pip install jiwer


  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.10.3.tar.gz (212 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [51 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-cpython-311
      creating build/lib.linux-x86_64-cpython-311/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.linux-x86_64-cpython-311/tokenizers
      creating build/lib.linux-x86_64-cpython-311/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib.linux-x86_64-cpython-311/tokenizers/models
      creating build/lib.linux-x86_64-cpython-311/tokenizers/decoders
    

In [1]:
from datasets import load_dataset, Dataset, Audio, load_from_disk
import numpy as np
import json
import os
import re
import glob
import numpy as np

wave_files = glob.glob('./ESC-50-master/audio/*.wav')
noise_dataset = Dataset.from_dict({'audio': wave_files}).cast_column("audio", Audio(sampling_rate=16_000))
test_dataset = load_from_disk("../timit_test_dataset")



In [2]:
import IPython.display as ipd
import numpy as np

rand_int =1
noise_signal = np.asarray(noise_dataset[30]["audio"]["array"])
signal_power = 10*np.log10(np.mean(noise_signal**2))
print(signal_power)
ipd.Audio(data=noise_signal , autoplay=True, rate=16000)

-31.986809844547977


In [7]:
test_signal = np.array(test_dataset[0]["input_values"])

# normalize signal power to 1 (0dB)
signal_power = np.mean(noise_signal**2)
noise_signal = noise_signal / np.sqrt(signal_power)

noise_amp_linear = 10**(10/20)
noise_signal = noise_signal*noise_amp_linear

signal_power_2 = 10*np.log10(np.mean(noise_signal**2))
print(signal_power_2)

if test_signal.shape[0] > noise_signal.shape[0]:
    padded_signal = np.zeros_like(test_signal)
    padded_signal[:noise_signal.shape[0]] = noise_signal
    corrupted_signal = test_signal+ padded_signal
else:
    noise_signal = noise_signal[:test_signal.shape[0]]
    corrupted_signal = test_signal + noise_signal

ipd.Audio(data=corrupted_signal , autoplay=True, rate=16000)


10.0


In [ ]:

np.random.seed(14)

def add_background_sound_to_signal(input_signal : np.ndarray, background_sound : np.ndarray, background_power_dB : float):

    # normalize signal power to 1 (0dB) and amplify by desired power afterwards
    signal_power = np.mean(background_sound**2)
    noise_signal = background_sound / np.sqrt(signal_power)
    noise_amp_linear = 10**(-background_power_dB/20)
    noise_signal = noise_signal*noise_amp_linear

    # pad or crop background sound and add to input signal
    if input_signal.shape[0] > noise_signal.shape[0]:
        padded_signal = np.zeros_like(input_signal)
        padded_signal[:noise_signal.shape[0]] = noise_signal
        corrupted_signal = input_signal + padded_signal
    else:
        noise_signal = noise_signal[:input_signal.shape[0]]
        corrupted_signal = input_signal + noise_signal
    return corrupted_signal 


noise_power_list = [-10,0,10,20]

def add_background_sound(batch):
    # batched output is "un-batched" to ensure mapping is correct
    input_values = np.array(batch["input_values"])

    sound_index = np.random.randint(0, len(noise_dataset))
    background_sound = np.array(noise_dataset[sound_index]["audio"]["array"])
    print(sound_index)
    for noise_power in noise_power_list:
        noisy_signal = add_background_sound_to_signal(input_values, background_sound, noise_power)
        batch[f"input_values_noisy_{int(noise_power)}"] = noisy_signal

    return batch

background_noise_test_dataset = test_dataset.map(add_background_sound)

In [10]:
background_noise_test_dataset.save_to_disk("/mount/arbeitsdaten/studenten1/team-lab-phonetics/2023/student_directories/tran/asr_timit/speech_datasets/timit_test_dataset_background_noisy")